<img src='../../media/common/LogoWekeo_Copernicus_RGB_0.png' align='left' height='96px'></img>

<hr>

# Analyzing Wet Snow Extent from SWS Time Series Based on Altitude and Local Aspect

This document includes scripts designed for the analysis of downloaded and extracted data on the local disk of a user.

This script is specifically developed to analyze the extent of wet snow from SWS time series data, considering factors such as altitude and local aspect. By using this script, users can gain valuable insights into the spatial and temporal patterns of wet snow distribution, which are crucial for various environmental and climatic studies.

## Environment Setup
Before we begin, we need to prepare our environment by installing and importing the necessary R packages.

### Load Required Libraries

In [ ]:
library(terra)
library(fs)
library(ggplot2)
library(viridis)
library(pracma)

library(magrittr)
library(dplyr)

library(hdar)

## Functions

#### Function to Open and Read GeoTIFF Files Assuming Extracted Products from the Catalog

In [ ]:
read_tif <- function(file_tif) {
  ds <- rast(file_tif)
  
  proj <- crs(ds)
  width <- ncol(ds)
  height <- nrow(ds)
  count <- nlyr(ds)
  meta <- list(
    min = minmax(ds)[1, ],
    max = minmax(ds)[2, ],
    mean = global(ds, fun = mean, na.rm = TRUE),
    sd = global(ds, fun = sd, na.rm = TRUE)
  )
  
  list(data = ds, width = width, height = height, count = count, meta = meta, proj = proj)
}

#### Function to Calculate Aspect from a Digital Elevation Model

In [ ]:
aspect <- function(dem) {
  z <- matrix(values(dem), nrow = nrow(dem), ncol = ncol(dem))

  grad <- gradient(z)
  x <- grad$X
  y <- grad$Y
  atan2(-x, y)
}

## Usage

#### Downloading SWS and DEM Data

To begin our analysis, we first need to download the necessary Snow Water Equivalent (SWS) and Digital Elevation Model (DEM) data. This can be accomplished using the HDA (Harmonized Data Access) client.  
The SWS data provides information on the extent of wet snow, while the DEM data offers detailed elevation information for the study area.  
By querying the HDA client with specific parameters such as dataset ID, observation period, and bounding box coordinates, we can retrieve and download the relevant datasets to our local directory for further processing and analysis.

In [ ]:
hda_client <- hdar::Client$new()

download_dir <- "../../data/download/snow-and-ice/products"
if (!fs::dir_exists(download_dir)) {
  fs::dir_create(download_dir)
}

query_sws <- jsonlite::toJSON(list(
  "dataset_id" = "EO:CRYO:DAT:HRSI:SWS",
  "startdate" = "2022-11-01T01:00:00Z",
  "enddate" = "2022-12-01T01:00:00Z",
  "bbox" = c(
      6.213305360358289,
      44.881040312330924,
      6.27014257275879,
      44.925714461959465
  )
), auto_unbox = TRUE)

query_dem <- jsonlite::toJSON(list(
  "dataset_id" = "EO:DEM:DAT:COP-DEM_GLO-30-DTED__2023_1",
  "bbox" = c(
      6.213305360358289,
      44.881040312330924,
      6.27014257275879,
      44.925714461959465
  )
), auto_unbox = TRUE)

tryCatch({
    matches <- hda_client$search(query_sws, limit = 3)
    matches$download(download_dir)

    matches <- hda_client$search(query_dem, limit = 1)
    matches$download(download_dir) 
}, error = function(e) {
    print(e)
})

#### Organizing Downloaded Files for Analysis

To facilitate the analysis, we extract the downloaded SWS and DEM files into dedicated directories. This organization ensures that all relevant data is readily accessible and systematically arranged, enhancing the efficiency of subsequent processing and analysis steps.

In [ ]:
prod_dir <- "../../data/processing/snow-and-ice/products"
if (!fs::dir_exists(prod_dir)) {
  fs::dir_create(prod_dir)
}

# Find all zip files
zip_files <- fs::dir_ls(download_dir, regexp = "SWS_.*\\.zip$")

# Create SWS directory if it doesn't exist
sws_dir <- fs::path(prod_dir, "SWS")
if (!fs::dir_exists(sws_dir)) {
    fs::dir_create(sws_dir)
}

# Unzip each file
for (zip_file in zip_files) {
    zip::unzip(zip_file, exdir = sws_dir)
}

In [ ]:
# Find all tar files
tar_files <- fs::dir_ls(download_dir, glob = "*.tar")

dem_dir <- fs::path(prod_dir, "DEM")
if (!fs::dir_exists(dem_dir)) {
    fs::dir_create(dem_dir)
}

# Extract each file
for (tar_file in tar_files) {
    untar(tar_file, exdir = dem_dir)
}

#### Converting DEM from 30m to 60m Resolution

In our analysis, we require a Digital Elevation Model (DEM) with a 60m resolution. However, a 60m DEM is not available for direct download from any source. The Copernicus DEM, which we are using, is available at a 30m resolution. To meet our requirements, we need to convert the 30m DEM to a 60m resolution.

In [ ]:
convert_and_resample <- function(input_dt2, output_tiff, resampled_tiff, x_res_deg, y_res_deg) {

    dt2_dataset <- rast(input_dt2)
    if (is.null(dt2_dataset)) {
        stop(paste("Could not open", input_dt2))
    }

    # Convert .dt2 to GeoTIFF
    writeRaster(dt2_dataset, output_tiff, overwrite = TRUE, filetype = "GTiff")

    tiff_dataset <- rast(output_tiff)
    if (is.null(tiff_dataset)) {
        stop(paste("Could not open", output_tiff))
    }

    # Create an empty raster with the desired resolution
    target_res <- c(x_res_deg, y_res_deg)
    template <- rast(tiff_dataset)
    res(template) <- target_res

    # Resample the GeoTIFF to the desired resolution
    resampled_dataset <- resample(tiff_dataset, template, method = "bilinear")

    if (is.null(resampled_dataset)) {
        stop(paste("Could not resample", output_tiff, "to", resampled_tiff))
    }

    writeRaster(resampled_dataset, resampled_tiff, overwrite = TRUE, filetype = "GTiff")

    print(paste("Successfully converted", input_dt2, "to", resampled_tiff, "with", x_res_deg, "x", y_res_deg, "degrees resolution."))
}

dt2_files <- fs::dir_ls(dem_dir, recurse = TRUE, regexp = ".*\\.dt2$")

resampled_dir <- fs::path(dem_dir, "resampled")
if (!fs::dir_exists(resampled_dir)) {
    fs::dir_create(resampled_dir)
}

# 60 meters to degrees conversion
x_res_deg <- 60 / 111320
y_res_deg <- 60 / 111320

# Convert and resample .dt2 files
for (dt2_file in dt2_files) {
    base_name <- path_ext_remove(path_file(dt2_file))
    output_tiff <- fs::path(resampled_dir, paste0(base_name, ".tif"))
    resampled_tiff <- fs::path(resampled_dir, paste0(base_name, "_60m.tif"))
    convert_and_resample(dt2_file, output_tiff, resampled_tiff, x_res_deg, y_res_deg)
}

In [ ]:
files_wsm <- fs::dir_ls(fs::path(prod_dir, "SWS"), recurse = TRUE, regexp = "SWS_.*_WSM\\.tif")
file_elev <- fs::dir_ls(fs::path(prod_dir, "DEM/resampled"), recurse = TRUE, regexp =  ".*DEM_60m\\.tif")

#### Open and Read the DEM File

In [ ]:
elev_data <- read_tif(file_elev[1])$data
elev_data[is.na(elev_data)] <- 255

#### Calculate the aspect from the DEM using the function defined in Box [3], and plot the resulting layer.
<table><tr><td>
Note: aspect calculation in the function provides the aspect in radians. For the illustration with the windrose toolbox, aspect information must be converted to degress, scaled between 0 and 360°. So, an offset of 180° is added to the conversion formula to rescale values from range [-180, 180] to [0, 360].
</td></tr></table>

In [ ]:
# Calculate aspect
asp_data <- aspect(elev_data)

# Convert aspect to degrees
aspdeg <- (180 / pi) * asp_data + 180
aspdeg[is.na(aspdeg)] <- 0
aspdeg <- rast(aspdeg)

# Plot the aspect data
asp_df <- as.data.frame(aspdeg, xy = TRUE)
colnames(asp_df) <- c("x", "y", "aspect")

options(repr.plot.width = 10, repr.plot.height = 8)
ggplot(asp_df, 
    aes(x = x, y = y, fill = aspect)) +
    geom_raster() +
    coord_fixed() +
    scale_fill_viridis_c(breaks = seq(0, 360, by = 50)) +
    scale_x_continuous(breaks = seq(min(asp_df$y), max(asp_df$y), by = 250)) +
    scale_y_continuous(breaks = seq(min(asp_df$x), max(asp_df$x), by = 250)) +
    theme_minimal() +
    theme(
        legend.text = element_text(size = 16), 
        legend.title = element_blank(),
        legend.key.height = unit(2, "cm"),
        axis.title.x = element_blank(),
        axis.title.y = element_blank()
    )

#### Open and read all Wet Snow products for the Mountains:

In [ ]:
wsm_datas <- list()

for (file_wsm in files_wsm) {
  wsm_datas <- append(wsm_datas, list(read_tif(file_wsm)$data))
}

#### Extract the aspect and elevation information for wet snow pixels and add this data to a pandas DataFrame for each SWS product in the time series:
<table><tr><td>
Note: You can activate <b>maxelev</b>, <b>minelev</b>, and <b>elevrange</b> if needed to obtain the minimum and maximum elevation, as well as the elevation range of the selected scene.
</td></tr></table>

In [ ]:
# Function to create and display wind rose plot for each data frame
create_windrose_plot <- function(df) {

  # Remove NA values
  df <- df %>%
    filter(!is.na(elevation) & !is.na(aspect))

  # Create bins for elevation
  df <- df %>%
    mutate(elevation_bin = cut(elevation, breaks = c(seq(0, 3500, by = 350), Inf))) #, include.lowest = TRUE))

  # Create a summary data frame for plotting
  df_summary <- df %>%
    group_by(aspect = cut(aspect, breaks = seq(-22.5, 360-22.5, by = 45), labels = c("N", "NE", "E", "SE", "S", "SW", "W", "NW"), include.lowest = TRUE), elevation_bin) %>%
    summarize(count = n(), .groups = 'drop') %>%
    filter(!is.na(aspect))

  ggplot(df_summary, aes(x = aspect, y = count, fill = elevation_bin)) +
    geom_col(position = "dodge") +
    coord_polar(start = -pi/8) +
    scale_x_discrete(drop = FALSE) +
    scale_fill_viridis_d() +
    theme_minimal() +
    labs(title = "", x = "Aspect (direction)", y = "Count") +
    theme(
      axis.title.x = element_blank(),
      axis.title.y = element_blank(),
      panel.grid.major = element_line(color = "grey80"),
      panel.grid.minor = element_blank(),
      panel.background = element_rect(fill = "white"),
      legend.key.width = unit(2, "cm"),  
      legend.text = element_text(size = 12),
      legend.title = element_blank()
    ) +
    scale_y_continuous(labels = scales::comma_format())
}

val_wet_snow <- 110
dataframe_list <- list()

for (i in seq_along(wsm_datas)) {
    wsm_data <- wsm_datas[[i]]

    sel <- (values(wsm_data) == val_wet_snow)
    sel[is.na(sel)] <- FALSE

    elev_sel <- elev_data[sel]
    asp_sel <- aspdeg[sel]

    df <- data.frame(elevation = elev_sel, aspect = asp_sel)
    colnames(df) <- c("elevation", "aspect")
    dataframe_list[[i]] <- df
}

for (df in dataframe_list) {
  if (length(df$elevation) > 0) {
   print(create_windrose_plot(df))
  }
}


***
><span style = "font-family:Verdana; font-size:0.7em">Copyright © <font color='darkblue'>2022</font>, by ENVEO IT GmbH.</span>  
<span style = "font-family:Verdana; font-size:0.7em">Contributors: Gabriele Schwaizer, Lars Keuris</span>  
<span style = "font-family:Verdana; font-size:0.7em">URL: www.enveo.at</span> 
***
<p style = "font-family:Verdana; font-size:0.7em; line-height:0.5">Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files, to use the Software without restriction.</p>  
<p style = "font-family:Verdana; font-size:0.7em; line-height:1.15; text-align:justify">THE SOFTWARE IS PROVIDED AS IS, WITHOUT WARRANTY OF ANYKIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. <b>IN NO EVENT SHALL THE COPYRIGHT HOLDER BE LIABLE FOR ANY CLAIM, DIRECT INDIRECT, SPECIAL, INCIDENTAL, OR CONSEQUENTIAL DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.</b></p>

*** 